In [36]:
# loading the libraries
import random
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import pooch
import tempfile
import cartopy.crs as ccrs
import holoviews
from geoviews import Dataset as gvDataset
import geoviews.feature as gf
from geoviews import Image as gvImage
import boto3
import botocore
from pythia_datasets import DATASETS
from scipy import stats

In [37]:
# helper functions
def pooch_load(filelocation=None,filename=None,processor=None):
    shared_location='/home/jovyan/shared/Data/Projects/Sea_Level' # this is different for each day
    user_temp_cache=tempfile.gettempdir()
    
    if os.path.exists(os.path.join(shared_location,filename)):
        file = os.path.join(shared_location,filename)
    else:
        file = pooch.retrieve(filelocation,known_hash=None,fname=os.path.join(user_temp_cache,filename),processor=processor)

    return file

In [ ]:
#loading the dataset and the sea surface height information
url_ECCO='~/shared/Data/Projects/Sea_Level/SEA_SURFACE_HEIGHT_mon_mean_1992-01-2017-12_ECCO_V4r4_latlon_0p50deg.nc'
ds=xr.open_dataset(url_ECCO)
ssh=ds["SSH"]
ssh

In [ ]:
#defining coordinates to cover the extent of Indonesia
minlat=-14
maxlat=10
minlon=89
maxlon=142

In [ ]:
#cropping the data for Indonesia
ssh=ssh.sel(latitude=slice(minlat, maxlat), longitude=slice(minlon, maxlon))

In [ ]:
#calculate the mean sea surface height across all years available - the data starts from 1992
ssh_mean=ssh.mean("time")
ssh_mean

In [ ]:
#plotting the mean ssh across all data available
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.Robinson(central_longitude=110))
ssh_mean.plot(transform=ccrs.PlateCarree(), ax=ax)

# add coastlines
ax.coastlines()

In [ ]:
#calculate the difference between mean SSH and each of these
ssh_year=ssh.groupby("time.year").mean()

#calculate the mean anomaly per year
ssh_anomaly=ssh_year-ssh_mean
ssh_anomaly

In [ ]:
holoviews.extension("bokeh")
dataset_plot = gvDataset(
    ssh_anomaly
)  # only the first 10, as it is a time consuming task
images = dataset_plot.to(gvImage, ["longitude", "latitude"], ["SSH"], "year")
images.opts(
    cmap="BrBG",
    colorbar=True,
    width=600,
    height=400,
    projection=ccrs.Robinson(),
    clabel="Yearly SSH Anomaly \n(mm/day)",
) * gf.coastline

In [ ]:
ssh_anomaly_mean = ssh_anomaly.mean(dim=["latitude", "longitude"])
ssh_anomaly_mean

In [ ]:
#plot spatial mean across latitude and longitude
ssh_anomaly_mean.sel(year=slice(2000,2014)).plot()

In [ ]:
#import data from CESM2 for sea surface temperature
filepath = DATASETS.fetch("CESM2_sst_data.nc")
ds2 = xr.open_dataset(filepath)
ds2


In [ ]:
#calculate sea surface temperature anomaly across Indonesia
indonesia_tos = ds2.tos.sel(lat = slice(minlat, maxlat
), lon = slice(minlon, maxlon))
indonesia_tos_mean = indonesia_tos.groupby("time.year").mean()
indonesia_tos_mean = indonesia_tos_mean.mean(dim=["lat","lon"])
indonesia_tos_mean_time = indonesia_tos.groupby("time.year").mean(dim=["lat","lon"]).mean(dim="time")
indonesia_tos_anom = indonesia_tos_mean - indonesia_tos_mean_time
indonesia_tos_anom

indonesia_tos_anom.plot()

In [ ]:


indo_tos_month = indonesia_tos.groupby('time.month')
indo_tos_month_mean = indo_tos_month.mean(dim='time')

indo_tos_jan_clim = indo_tos_month_mean[0].mean(dim=['lat','lon'])

indo_tos_jan = indo_tos_month[1].mean(dim=['lat','lon'])
jan_anomaly_tos = indo_tos_jan - indo_tos_jan_clim


ssh_month = ssh.groupby('time.month')
ssh_month_mean = ssh_month.mean(dim='time')
ssh_jan_clim = ssh_month_mean[0].mean(dim=['latitude', 'longitude'])
ssh_jan = ssh_month[1].mean(dim=['latitude', 'longitude'])
ssh_jan_anomaly = ssh_jan - ssh_jan_clim

fig, axs = plt.subplots(2, sharex=True)
fig.suptitle("January: SSH anomaly vs. TOS anomaly")
axs[0].plot(ssh_jan_anomaly[8:-3])
axs[0].set_ylabel("SSH (m)")
axs[0].axhline(y=0, color="k", linestyle="-")
axs[1].plot(jan_anomaly_tos)
axs[1].set_ylabel("SST anomaly (deg C)")
axs[1].set_xlabel("Time")
axs[1].axhline(y=0, color="k", linestyle="-")

#ssh_anomaly_mean.sel(year=slice(2000,2014))#.plot()
#year 0 is 2000

In [ ]:
#Pearsons correlation coefficient for SSH and TOS
ssh_tos_r, ssh_tos_p = stats.pearsonr(ssh_jan_anomaly[8:-3], jan_anomaly_tos)
print("SSH-TOS Corr Coef: " + str(ssh_tos_r) + ", p-val: " + str(ssh_tos_p))


In [ ]:

indo_tos_month_mean = indo_tos_month.mean(dim='time')

indo_tos_jan_clim = indo_tos_month_mean[0].mean(dim=['lat','lon'])

indo_tos_jan = indo_tos_month[1].mean(dim=['lat','lon'])
jan_anomaly_tos = indo_tos_jan - indo_tos_jan_clim


ssh_month = ssh.groupby('time.month')
ssh_month_mean = ssh_month.mean(dim='time')
ssh_jan_clim = ssh_month_mean[0].mean(dim=['latitude', 'longitude'])
ssh_jan = ssh_month[1].mean(dim=['latitude', 'longitude'])
ssh_jan_anomaly = ssh_jan - ssh_jan_clim

fig, axs = plt.subplots(2, sharex=True)
fig.suptitle("January: SSH anomaly vs. TOS anomaly")
axs[0].plot(jan_anomaly_tos.time, ssh_jan_anomaly[8:-3])
axs[0].set_ylabel("SSH (m)")
axs[0].axhline(y=0, color="k", linestyle="-")
axs[1].plot(
    jan_anomaly_tos.time, jan_anomaly_tos)
axs[1].set_ylabel("SST anomaly (deg C)")
axs[1].set_xlabel("Time")
axs[1].axhline(y=0, color="k", linestyle="-")

#ssh_anomaly_mean.sel(year=slice(2000,2014))#.plot()
#year 0 is 2000

In [ ]:
# set up two subplots that share the x-axis to compare monthly precipitation and monthly anomaly
fig, axs = plt.subplots(2, sharex=True)
fig.suptitle("SSH anomaly vs. TOS anomaly (yearly)")
axs[0].plot(indonesia_tos_anom.year, ssh_anomaly_mean.sel(year=slice(2000,2014)))
axs[0].set_ylabel("SSH (m)")
axs[1].plot(
    indonesia_tos_anom.year, indonesia_tos_anom)
axs[1].set_ylabel("SST anomaly (deg C)")
axs[1].set_xlabel("Time")


In [ ]:
#Pearsons correlation coefficient for SSH and TOS
ssh_tos_r, ssh_tos_p = stats.pearsonr(ssh_anomaly_mean.sel(year=slice(2000,2014)), indonesia_tos_anom)
print("SSH-TOS Corr Coef: " + str(ssh_tos_r) + ", p-val: " + str(ssh_tos_p))

# Precipitation

In [ ]:
# imports
import s3fs
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import boto3
import botocore
import os
import pooch
import tempfile
import holoviews
from geoviews import Dataset as gvDataset
import geoviews.feature as gf
from geoviews import Image as gvImage

# @title Helper functions

def pooch_load(filelocation=None, filename=None, processor=None):
    shared_location = "/home/jovyan/shared/data/tutorials/W1D3_RemoteSensingLandOceanandAtmosphere"  # this is different for each day
    user_temp_cache = tempfile.gettempdir()

    if os.path.exists(os.path.join(shared_location, filename)):
        file = os.path.join(shared_location, filename)
    else:
        file = pooch.retrieve(
            filelocation,
            known_hash=None,
            fname=os.path.join(user_temp_cache, filename),
            processor=processor,
        )

    return file

# connect to the AWS S3 bucket for the GPCP Monthly Precipitation CDR data
fs = s3fs.S3FileSystem(anon=True)

# get the list of all data files in the AWS S3 bucket
file_pattern = "noaa-cdr-precip-gpcp-monthly-pds/data/*/gpcp_v02r03_monthly_*.nc"
file_location = fs.glob(file_pattern)

# open connection to all data files
client = boto3.client(
    "s3", config=botocore.client.Config(signature_version=botocore.UNSIGNED)
)  # initialize aws s3 bucket client
file_ob = [
    pooch_load(filelocation="http://s3.amazonaws.com/" + file, filename=file)
    for file in file_location
]

# open all the monthly data files and concatenate them along the time dimension
# this process will take ~ 1 minute to complete due to the number of data files
ds = xr.open_mfdataset(file_ob, combine="nested", concat_dim="time")

# comment for colab users only: this could toss an error message for you. 
# you should still be able to use the dataset with this error just not print ds
# you can try uncommenting the following line to avoid the error
# ds.attrs['history']='' # the history attribute have unique chars that cause a crash on Google colab. 
ds

In [ ]:

precip = ds.precip.sel(latitude=slice(-14, 10), longitude=slice(89, 142), time=slice("2000","2014"))
precip_clim= precip.groupby("time.year").mean()
precip_clim=precip_clim.mean(dim=["latitude","longitude"])
precip_clim_mean = precip.mean(dim="time").mean(dim=["latitude","longitude"])
precip_clim_anomaly=precip_clim - precip_clim_mean #subtract the clim
precip_clim_anomaly.plot()

In [ ]:
#Pearsons correlation coefficient for PRECIP and SSH
precip_ssh_r, precip_ssh_p= stats.pearsonr(ssh_anomaly_mean.sel(year=slice(2000,2014)), precip_clim_anomaly)
print("Precipitation-SSH Corr Coef: " + str(precip_ssh_r) + ", p-val: " + str(precip_ssh_p))

In [ ]:
#Pearsons correlation coefficient for PRECIP and TOS
precip_tos_r, precip_tos_p= stats.pearsonr(indonesia_tos_anom, precip_clim_anomaly)
print("Precipitation-SSH Corr Coef: " + str(precip_tos_r) + ", p-val: " + str(precip_tos_p))


In [ ]:
ssh_anomaly_mean

In [ ]:
indonesia_tos_anom.plot()
precip_clim_anomaly.plot()
ssh_anomaly_mean.sel(year=slice(2000,2014)).plot()

In [ ]:

fig, axs = plt.subplots(3, sharex=True)
fig.suptitle("January: SSH anomaly vs. TOS anomaly")
axs[0].plot(ssh_jan_anomaly[8:-3])
axs[0].set_ylabel("SSH (m)")
axs[0].set_xlabel("Time")
axs[0].axhline(y=0, color="k", linestyle="-")
axs[1].plot(jan_anomaly_tos)
axs[1].set_xlabel("Time")
axs[1].set_ylabel("TOS anomaly (degC)")
axs[1].axhline(y=0, color="k", linestyle="-")
axs[2].plot(precip_clim_anomaly.time, precip_clim_anomaly)
axs[2].set_xlabel("Time")
axs[2].set_ylabel("Precipitation anomaly (mm)")
axs[2].axhline(y=0, color="k", linestyle="-")


In [ ]:
# URLS for daily data

# Pandang: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d107.nc 
# Bitung: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d033.nc
# Siboga: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d122.nc
# Sabang: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d123.nc
# Prigi: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d125.nc
# Ambon: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d133.nc
# Cilacap: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d162.nc
# Benoa: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d163.nc
# Tanjung: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d416.nc
# Sedang: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d417.nc
# Waikelo: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d418.nc
# Lembar: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d419.nc
# Saumlaki: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d420.nc
# Telukdalam: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d913.nc
# Meulaboh: https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d914.nc

dict_of_chosen_locations={'Pandang': 'https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d107.nc',
                          'Bitung': 'https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d033.nc',
                          'Sabang': 'https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d123.nc',
                          'Prigi': 'https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d125.nc',
                          'Ambon': 'https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d133.nc',
                          'Lembar': 'https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d419.nc',
                          'Siboga': 'https://uhslc.soest.hawaii.edu/data/netcdf/fast/daily/d122.nc',
                         }

In [ ]:
#takes the global variables so that we can assign the name "ds_<enter location>" to each data set
my_var=globals()
list_of_sea_level_datasets = []
# loop through the dictionary of chosen locations to assign the netcdf data to each name and store it in global variables
for k,v in dict_of_chosen_locations.items():
    my_var["ds_"+k.lower()] = xr.open_dataset(
    pooch.retrieve(v, known_hash=None)
    )
    list_of_sea_level_datasets.append(eval(("ds_"+k.lower()))) #creates a list for sea level datasets for future use

# calculate the monthly mean between a time period for the region
# we can see that the sea level differs greatly from summer to winter seasons
list_of_sl_mean= []
ax = plt.axes()
for i, key in enumerate(dict_of_chosen_locations): #i represents the loop count and key is the name of the location.
    my_var[key.lower()+"_mean"]=(
    list_of_sea_level_datasets[i].sea_level.sel(time=slice("2008-10-02", "2021-12-11")).groupby('time.year').mean()
    )
    list_of_sl_mean.append(eval(key.lower()+"_mean")) #creates a list for mean sea level over time for future use
    eval(key.lower()+"_mean").plot(ax=ax, label = key) #plots mean for each location
    
ax.legend()

In [ ]:
ffilled_datasets = [x.ffill(dim="time") for x in list_of_sea_level_datasets]
keys = list(dict_of_chosen_locations)
for i, a in enumerate(keys):
    
    keys.remove(a)
    for j, b in enumerate(keys):
        r,p=stats.pearsonr(ffilled_datasets[i].sea_level.sel(time=slice("2008-10-02", "2021-12-11")).squeeze(), ffilled_datasets[j].sea_level.sel(time=slice("2008-10-02", "2021-12-11")).squeeze())
        print(a,b,r,p)

In [ ]:
loc_lat=[-0.9397466858685701, 1.48, 5.881303082560345, -8.269878225923001, -3.6368986779246977, -8.73904591353275, 1.75]
loc_lon=[100.4159027330682, 125.55, 95.33574428146883, 111.72181131396911, 128.16884101601298, 116.09401919927895, 98.76]

In [ ]:
# Plot SSH, TOS, Tidal Gauge, Precip Data for all 7 locations
start = "2008-10-02"
end = "2017-01-01" #Use specific dates because some locations don't start on Jan 1st of 2008
tos = ds2.tos

fig, axes = plt.subplots(7,4,figsize=(16,12))
# loop over locations 
for i,a in enumerate(list(dict_of_chosen_locations)):
    ssh_loc = ssh.sel(latitude=loc_lat[i], longitude=loc_lon[i], method = "nearest")
    ssh_loc_mean = ssh_loc.sel(time=slice(start, end)).groupby("time.year").mean()
    ssh_loc_anom = ssh_loc_mean - ssh_loc_mean.mean(dim="time")
    tos_loc = tos.sel(lat=loc_lat[i], lon=loc_lon[i], method = "nearest")
    tos_loc_mean = tos_loc.groupby("time.year").mean()
    precip_loc = ds.precip.sel(latitude=loc_lat[i], longitude=loc_lon[i], method = "nearest")
    precip_loc_mean = precip_loc.groupby("time.year").mean()
    tidal_gauge_loc = ffilled_datasets[i].sea_level.sel(time=slice(start,end)).squeeze()
    tidal_gauge_loc_mean = tidal_gauge_loc.groupby("time.year").mean()
    axes[i,0].plot(ssh_loc_anom)
    axes[i,1].plot(tos_loc_mean)
    axes[i,2].plot(tidal_gauge_loc_mean)
    axes[i,3].plot(precip_loc_mean)
    axes[i,0].set_xlabel("SSH")
    axes[i,1].set_xlabel("TOS")
    axes[i,2].set_xlabel("Tidal Gauge")
    axes[i,3].set_xlabel("Precip")

In [ ]:
#corr coeff for precip vs. tidal gauge for all six locations
tos = ds2.tos
list_of_corr_coeff_precip_vs_tidal_gauge = []
for i,a in enumerate(list(dict_of_chosen_locations)):
    
    
    b = ds.precip.sel(latitude=loc_lat[i], longitude=loc_lon[i], method = "nearest").ffill("time")
    c = ffilled_datasets[i].sea_level.sel(time=slice(start,end)).squeeze()
    min_time = max(min(c.time), min(b.time)) #these two lines take the maximum overlap of time for the two different locations.
    max_time = min(max(c.time), max(b.time))
    c_new=c.sel(time = slice(min_time, max_time)).groupby("time.year").mean()
    b_new=b.sel(time = slice(min_time, max_time)).groupby("time.year").mean()
    r, p = stats.pearsonr(c_new,b_new)
    list_of_corr_coeff_precip_vs_tidal_gauge.append([r,p])
    print("Corr-Coeff: ",r," p-value: ",p)

In [ ]:
for i in dict_of_chosen_locations.keys():
    print(i)